# 🤖↔️🤖 Opdracht 3: AI's met Elkaar Laten Praten

## 🎯 Wat gaan we doen?

In dit notebook gaan we meerdere AI's maken die met elkaar gaan praten! Je leert:
- Verschillende AI persoonlijkheden tegelijk runnen
- AI's automatisch met elkaar laten converseren
- Conversatie loops bouwen
- Grappige, interessante of bizarre gesprekken creëren

👉 Antwoordenblad: zie [antwoorden.md](antwoorden.md) voor voorbeeldantwoorden.

## 🎭 Het Concept

**Idee:** Maak verschillende AI bots met unieke persoonlijkheden en laat ze met elkaar chatten!

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load the .env file from the same directory as the notebook
load_dotenv()

# Get configuration from environment variables
API_KEY = os.getenv('API_KEY', 'ollama')
BASE_URL = os.getenv('BASE_URL', 'http://localhost:11434/v1')
MODEL = os.getenv('MODEL', 'llama2')

print(f"🔗 Configuratie: {BASE_URL}")
print(f"🔑 API Key: {'*' * 8 + API_KEY[-4:] if API_KEY and len(API_KEY) > 4 else 'Niet ingesteld'}")
print(f"🤖 Model: {MODEL}")


In [ ]:
from openai import OpenAI

# Initialise de AI client
client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)


## 🧑‍💼 AI Character Class

Eerst maken we een class voor AI karakters:

In [ ]:
class AICharacter:
    def __init__(self, name, personality, emoji="🤖", model=None):
        self.name = name
        self.emoji = emoji
        self.personality = personality
        self.model = model or MODEL  # Optioneel eigen model per karakter
        self.conversation_history = []
    
    def respond_to(self, message, speaker_name="Someone"):
        """Genereer een response op een bericht"""
        
        # Bouw de prompt
        messages = [
            {
                "role": "system", 
                "content": f"Je naam is {self.name}. {self.personality} Houd je antwoorden kort (max 2 zinnen)."
            }
        ]
        
        # Voeg recente conversatie geschiedenis toe
        recent_history = self.conversation_history[-4:]  # Laatste 4 berichten
        messages.extend(recent_history)
        
        # Voeg het nieuwe bericht toe
        messages.append({
            "role": "user",
            "content": f"{speaker_name} zegt: {message}"
        })
        
        try:
            # API call
            response = client.chat.completions.create(
                model=self.model,
                messages=messages,
                max_tokens=100,
                temperature=0.8  # Wat creatiever voor interessante gesprekken
            )
            
            ai_response = response.choices[0].message.content
            
            # Voeg toe aan geschiedenis
            self.conversation_history.append({
                "role": "user",
                "content": f"{speaker_name}: {message}"
            })
            self.conversation_history.append({
                "role": "assistant",
                "content": ai_response
            })
            
            return ai_response
            
        except Exception as e:
            return f"[ERROR: {e}]"
    
    def __str__(self):
        return f"{self.emoji} {self.name}"

# Test de class
piraat = AICharacter(
    name="Captain Blackbeard",
    personality="Je bent een stoere piraat! Praat altijd als een piraat met 'arr' en 'matey'.",
    emoji="🏴‍☠️"
)

print(f"Karakter gemaakt: {piraat}")
print(f"Test response: {piraat.respond_to('Hallo!', 'Test User')}")

## 🎪 Verschillende Karakters Maken

Laten we een hele cast van karakters maken:

In [ ]:
# Maak verschillende AI karakters
karakters = {
    "piraat": AICharacter(
        name="Captain Jack",
        personality="Je bent een stoere piraat! Praat met 'arr', 'matey' en vertel over de zee!",
        emoji="🏴‍☠️"
    ),
    
    "robot": AICharacter(
        name="R2-D2000",
        personality="Je bent een robot. Praat mechanisch met 'BEEP BOOP'. Analyseer alles logisch.",
        emoji="🤖"
    ),
    
    "chef": AICharacter(
        name="Chef Mario",
        personality="Je bent een Italiaanse chef! Praat over eten, gebruik 'mama mia!' en Italiaanse woorden.",
        emoji="👨‍🍳"
    ),
    
    "detective": AICharacter(
        name="Sherlock AI",
        personality="Je bent een detective. Alles is mysterieus! Zoek naar aanwijzingen en complotten.",
        emoji="🔍"
    ),
    
    "alien": AICharacter(
        name="Zorp",
        personality="Je bent een alien die net op aarde is geland. Alles is vreemd! Stel veel vragen over mensen.",
        emoji="👽"
    )
}

# Toon alle karakters
print("🎭 AI Cast:")
for key, karakter in karakters.items():
    print(f"  {karakter}: {karakter.personality[:50]}...")

## 🗣️ Twee AI's Laten Praten

Nu gaan we twee AI's een gesprek laten voeren:

In [ ]:
def ai_conversation(ai1, ai2, starter_message, rounds=5):
    """Laat twee AI's met elkaar praten"""
    print(f"🎬 Conversatie tussen {ai1} en {ai2}")
    print("=" * 50)
    
    current_message = starter_message
    current_speaker = ai1
    other_speaker = ai2
    
    print(f"📢 Starter: {starter_message}")
    print()
    
    for round_num in range(rounds):
        # AI reageert op het bericht
        response = current_speaker.respond_to(
            current_message, 
            other_speaker.name
        )
        
        print(f"{current_speaker}: {response}")
        print()
        
        # Wissel van spreker
        current_message = response
        current_speaker, other_speaker = other_speaker, current_speaker
        
        # Kleine pauze voor dramaturgie
        time.sleep(0.5)
    
    print("🎬 [END SCENE]")
    print()

# Test: Piraat vs Robot
ai_conversation(
    karakters["piraat"], 
    karakters["robot"], 
    "Wat is het beste voertuig voor reizen?",
    rounds=10
)

## 💡 EXPERIMENTEER TIJD!

**JOUW BEURT:** Probeer verschillende combinaties van AI's!

In [ ]:
# EXPERIMENTEER: Probeer verschillende combinaties!

# Voorbeeld 1: Chef vs Alien
print("🍝👽 Chef vs Alien over eten:")
ai_conversation(
    karakters["chef"], 
    karakters["alien"], 
    "Wat is het beste eten ter wereld?",
    rounds=3
)

print("\n" + "="*60 + "\n")

# Voorbeeld 2: Detective vs Piraat
print("🔍🏴‍☠️ Detective vs Piraat over misdaad:")
ai_conversation(
    karakters["detective"], 
    karakters["piraat"], 
    "Er is iets mysterieus aan de hand...",
    rounds=3
)

# JOUW EXPERIMENTEN HIER:
# Probeer:
# - karakters["robot"] vs karakters["alien"]
# - Een discussie over "Wat is liefde?"
# - Een debate over "Pineapple op pizza - ja of nee?"


## 🎪 Multi-AI Group Chat

Wat als we meer dan 2 AI's laten praten? Group chat!

In [ ]:
import random

def ai_group_chat(ai_list, topic, rounds=8):
    """Laat meerdere AI's in een groepsgesprek"""
    print(f"🎪 GROUP CHAT over: {topic}")
    print(f"👥 Deelnemers: {', '.join([str(ai) for ai in ai_list])}")
    print("=" * 60)
    
    current_message = topic
    conversation_log = []
    
    for round_num in range(rounds):
        # Kies willekeurig welke AI mag praten
        current_ai = random.choice(ai_list)
        
        # Bepaal wie de laatste spreker was (voor context)
        last_speaker = "Someone" if not conversation_log else conversation_log[-1]["speaker"]
        
        # AI reageert
        response = current_ai.respond_to(current_message, last_speaker)
        
        print(f"{current_ai}: {response}")
        print()
        
        # Update voor volgende ronde
        current_message = response
        conversation_log.append({
            "speaker": current_ai.name,
            "message": response
        })
        
        time.sleep(0.3)
    
    print("🎪 [GROUP CHAT ENDED]")
    return conversation_log

# Test group chat met 4 AI's
group_members = [
    karakters["piraat"],
    karakters["robot"], 
    karakters["chef"],
    karakters["detective"]
]

chat_log = ai_group_chat(
    group_members, 
    "Wat is het perfecte weekend?",
    rounds=16
)

## 🎯 Gestructureerde Debates

Laten we een formeel debate organiseren tussen AI's:

In [ ]:
def ai_debate(ai_pro, ai_contra, topic, rounds=3):
    """Organiseer een formeel debate tussen twee AI's"""
    print(f"⚖️ DEBATE: {topic}")
    print(f"👍 PRO: {ai_pro}")
    print(f"👎 CONTRA: {ai_contra}")
    print("=" * 60)
    
    # Pro begint
    pro_message = ai_pro.respond_to(
        f"Geef argumenten VOOR: {topic}", 
        "Debate Moderator"
    )
    print(f"👍 {ai_pro}: {pro_message}")
    print()
    
    current_message = pro_message
    
    for round_num in range(rounds):
        # Contra reageert
        contra_response = ai_contra.respond_to(
            f"Geef argumenten TEGEN wat {ai_pro.name} zei: {current_message}",
            ai_pro.name
        )
        print(f"👎 {ai_contra}: {contra_response}")
        print()
        
        # Pro reageert terug
        if round_num < rounds - 1:  # Niet in laatste ronde
            pro_response = ai_pro.respond_to(
                f"Reageer op dit argument: {contra_response}",
                ai_contra.name
            )
            print(f"👍 {ai_pro}: {pro_response}")
            print()
            current_message = pro_response
        
        time.sleep(0.5)
    
    print("⚖️ [DEBATE ENDED - Wie won volgens jou?]")

# Test debates
print("🍕 Debate 1: Pineapple op pizza")
ai_debate(
    karakters["chef"],      # PRO pineapple
    karakters["robot"],     # CONTRA pineapple  
    "Ananas hoort op pizza",
    rounds=2
)

print("\n" + "="*60 + "\n")

print("🌊 Debate 2: Leven op zee vs land")
ai_debate(
    karakters["piraat"],    # PRO zeeleven
    karakters["detective"], # CONTRA zeeleven
    "Het is beter om op zee te leven dan op land",
    rounds=2
)

## 🎲 Random Scenario Generator

Laten we willekeurige scenario's maken voor AI gesprekken:

In [ ]:
import random

# Random scenario generator
scenarios = [
    "Jullie zijn gestrand op een onbewoond eiland",
    "Jullie moeten samen een nieuw restaurant openen", 
    "Er is een mysterieuze schat verstopt in de stad",
    "Aliens landen op aarde en jullie moeten ze verwelkomen",
    "Jullie zijn de laatste twee mensen/robots op aarde",
    "Er moet een nieuwe koning/koningin gekozen worden",
    "Jullie organiseren het perfecte feest",
    "Een tijdmachine is uitgevonden - wat nu?",
    "Jullie moeten een heel moeilijk raadsel oplossen",
    "De wereld heeft een nieuwe superheld nodig"
]

discussion_topics = [
    "Wat is geluk?",
    "Zijn robots beter dan mensen?",
    "Wat is de beste uitvinding ooit?",
    "Hoe ziet de perfecte wereld eruit?",
    "Waarom bestaan we?",
    "Is het universum bewust?",
    "Wat gebeurt er na de dood?",
    "Zijn we alleen in het universum?"
]

def random_ai_encounter():
    """Genereer een willekeurig AI gesprek"""
    # Kies willekeurige AI's
    ai_keys = list(karakters.keys())
    chosen_ais = random.sample(ai_keys, 2)
    ai1, ai2 = karakters[chosen_ais[0]], karakters[chosen_ais[1]]
    
    # Kies willekeurig scenario of topic
    if random.choice([True, False]):
        prompt = random.choice(scenarios)
        prompt_type = "SCENARIO"
    else:
        prompt = random.choice(discussion_topics)
        prompt_type = "DISCUSSIE"
    
    print(f"🎲 WILLEKEURIGE {prompt_type}")
    print(f"🎭 Karakters: {ai1} vs {ai2}")
    print(f"📜 {prompt_type.title()}: {prompt}")
    print()
    
    # Start gesprek
    ai_conversation(ai1, ai2, prompt, rounds=4)

# Genereer 2 willekeurige encounters
for i in range(2):
    print(f"\n{'='*60}")
    print(f"🎲 RANDOM ENCOUNTER #{i+1}")
    print('='*60)
    random_ai_encounter()
    print()

## 📝 JOUW GROTE OPDRACHT

**Opdracht:** Maak een korte scene waarin meerdere AI's met elkaar praten — elk met een eigen persoonlijkheid én een eigen model.

**Vereisten:**
1. Maak minstens 3 eigen `AICharacter`'s met unieke persoonlijkheden
2. Geef ieder karakter een eigen model via de `model=` parameter (bijv. met `MODEL_CHEF`, `MODEL_ROBOT`, `MODEL_DETECTIVE`)
3. Laat ze met elkaar praten in 1-op-1 en/of een groepsgesprek
4. Print de transcripties in de output

**Let op:**
- Houd antwoorden kort en speels (al ingesteld in de system prompt)
- Gebruik `ai_conversation` en/of `ai_group_chat`


In [ ]:
# 🚀 START HIER: Je eigen karakters + modellen

# Tip: Zet in je .env bijv. MODEL_CHEF, MODEL_ROBOT, MODEL_DETECTIVE
# of geef direct een model mee in de constructor.

# Voorbeeld (pas aan naar jouw idee):
#mijn_karakters = {
#    "chef": AICharacter(
#        name="Chef Bella",
#        personality="Je bent een enthousiaste Italiaanse chef. Gebruik veel culinaire termen.",
#        emoji="👩‍🍳",
#        model=MODEL_CHEF
#    ),
#    "robot": AICharacter(
#        name="Unit-42",
#        personality="Je bent een logische robot. Kort en analytisch.",
#        emoji="🤖",
#        model=MODEL_ROBOT
#    ),
#    "detective": AICharacter(
#        name="Inspector Pixel",
#        personality="Je bent een speurneus. Stel vragen en trek conclusies.",
#        emoji="🕵️",
#        model=MODEL_DETECTIVE
#    ),
#}

# ————————————————
# 1-op-1 voorbeeld (pas aan / kopieer voor andere paren)
#ai_conversation(mijn_karakters["chef"], mijn_karakters["robot"], "Wat is het beste gerecht ooit?", rounds=3)

# ————————————————
# Groepschat voorbeeld
#ai_group_chat(list(mijn_karakters.values()), "We openen samen een nieuw restaurant — wat is het concept?", rounds=5)


## ✅ Check Jezelf

Kun je deze vragen beantwoorden?

1. **Hoe voorkom je dat AI's exact hetzelfde zeggen in gesprekken?**
2. **Waarom is `temperature=0.8` beter voor AI gesprekken dan `temperature=0.1`?**
3. **Hoe kun je een AI emotioneler maken tijdens gesprekken?**
4. **Wat maakt een AI gesprek interessant om naar te kijken?**

Schrijf je antwoorden hieronder:

**Mijn antwoorden:**

1. Verschillende responses: 
2. Temperature voor gesprekken: 
3. Emotionele AI: 
4. Interessante gesprekken: 

## 🎉 Volgende Stap

Fantastisch! Je hebt AI's leren maken die met elkaar kunnen praten. In het volgende notebook gaan we function calling toevoegen - dan kunnen je AI's echte acties uitvoeren!

**Ga naar: [04-function-calling.ipynb](04-function-calling.ipynb)**

---

## 💡 Tips voor Thuis

- **Verschillende persoonlijkheden** maken gesprekken interessanter
- **Korte antwoorden** houden gesprekken dynamisch
- **Temperature 0.7-0.9** geeft creativere responses
- **Experimenteer met rare combinaties** - soms zijn die het leukst!
- **Bewaar interessante gesprekken** om later te bekijken